In [ ]:
import  music21 as ms21
import numpy as np
import os

midi_predict_path = "./POLYF_EXP/visual+caption_att_4_midi/"
midi_gt_path = "./POLYF_EXP/gt_midi/"

midi_predict_files = os.listdir(midi_predict_path)

In [ ]:
def process_midi(path):
    s=ms21.converter.parse(path)
    note2id = {
        "C": 0, 
        "C#": 1, 
        "D": 2, 
        "E-": 3, 
        "E": 4, 
        "F": 5, 
        "F#": 6, 
        "G": 7, 
        "G#": 8, 
        "A": 9, 
        "B-": 10,
        "B": 11, 
    }

    notes, bar, pattern, pattern_bar = [], [], [], [0 for k in range(16)]
    lastoffset = 0
    i, j = 1, 0

    for note in s.flat.notesAndRests:
        if isinstance(note, ms21.note.Rest):
            continue

        if note.offset >= 32 * i and lastoffset < 32 * i:
            notes.append(bar)
            bar = []
            i += 1
        while note.offset >= 4 * (j + 1) and lastoffset < 4 * (j + 1):
            pattern.append(np.array(pattern_bar))
            pattern_bar = [0 for k in range(16)]
            j += 1

        if isinstance(note,ms21.note.Note):
            # print(note.name, note.octave, note.pitch, note.pitch.midi, note.duration.quarterLength)
            bar.append(note2id[note.name])
            pattern_bar[int(4*(note.offset-4*j))] = 1
        else:
            try:
                for c_note in note.notes:
                    # print(c_note.name, c_note.pitch.midi, c_note.duration.quarterLength)
                    bar.append(note2id[c_note.name])
            except:
                pass
            pattern_bar[int(4*(note.offset-4*j))] = 1
            
        lastoffset = note.offset
    
    return notes, bar, pattern, pattern_bar

calculate pitch_class_histogram_entropy (PCHE)

In [ ]:
def pitch_class_histogram_entropy(notes):
    
    result = []

    for bar in notes:
        # Construct the 12-dimensional pitch class histogram
        histogram = np.zeros(12)
        for note in bar:
            pitch_class = note % 12
            histogram[pitch_class] += 1

        # Normalize the histogram
        histogram = histogram / np.sum(histogram)

        # Calculate the entropy
        entropy = -np.sum(histogram * np.log2(histogram + 1e-6))  # Added epsilon to avoid log(0)
        result.append(entropy)

    return sum(result)/len(notes)

pche = []

for item in midi_predict_files:
# for item in midi_gt_files:

    file_path = f'{midi_gt_path}{item}'

    notes, bar, pattern, pattern_bar = process_midi(file_path)

    pche.append(pitch_class_histogram_entropy(notes))

sum(pche) / len(pche)

calculate grooving_pattern_similarity (GPS)

In [ ]:
def grooving_pattern_similarity(g_a, g_b):
    assert len(g_a) == len(g_b), "Grooving patterns must have the same length"
    Q = len(g_a)
    gs = 1 - (1/Q) * np.sum(np.bitwise_xor(g_a, g_b))
    return gs

def cal_cps(pattern):

    results = []
    for i in range(len(pattern)):
        for j in range(i + 1, len(pattern)):
            g_a, g_b = pattern[i], pattern[j]
            results.append(grooving_pattern_similarity(g_a, g_b))

    return sum(results) / len(results)

cps = []

for item in midi_predict_files:

    file_path = f'{midi_predict_path}{item}'

    notes, bar, pattern, pattern_bar = process_midi(file_path)

    cps.append(cal_cps(pattern))

sum(cps) / len(cps)